<a href="https://colab.research.google.com/github/LOPAMUDRA-MUKHERJEE/DataSC/blob/main/AirTrafficFinal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'air-traffic-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F270572%2F561627%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240722%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240722T172747Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D1bc0ba6ab2f4acc39188dad00e9675c853b65178a521878b75c727fd35f871afbdf6d9512ea42cde206381af817e13792d6dc6109c77a031c00d94b2b58a3fae1bfb320c22656bc30c9ff77ba09c48a05b4c953b8ddda747b20ad2056e025fb59efb64c920bd9f331c7109118cb85eec5ec6bf07065e4bab1df17ea9dff400a4b2f0e9ec9bca7acebdc5898765f4c8d83b04e5f5282eee52b2d6f7a8aa5831b2718db32929c04799ee1bd26424b98cda79cc826ddc13f1889b9316422e8fec2c12964830818cbe4e92ca557aa46a0a571b59e226d8d3c5583946e76155fce9732a21f6168b94faa71e23615a2d7b211bec5361000e77697188a554fb3ae13857'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 176707 bytes downloaded
Downloaded and uncompressed: air-traffic-data
Data source import complete.


In [ ]:
import numpy as np # linear algebra
import pandas as pd
df= pd.read_csv('/kaggle/input/air-traffic-data/air_traffic_data.csv')
df.head(5)

,Activity Period,Operating Airline,Operating Airline IATA Code,Published Airline,Published Airline IATA Code,GEO Summary,GEO Region,Activity Type Code,Price Category Code,Terminal,Boarding Area,Passenger Count,Adjusted Activity Type Code,Adjusted Passenger Count,Year,Month
0,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Deplaned,Low Fare,Terminal 1,B,27271,Deplaned,27271,2005,July
1,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Enplaned,Low Fare,Terminal 1,B,29131,Enplaned,29131,2005,July
2,200507,ATA Airlines,TZ,ATA Airlines,TZ,Domestic,US,Thru / Transit,Low Fare,Terminal 1,B,5415,Thru / Transit * 2,10830,2005,July
3,200507,Air Canada,AC,Air Canada,AC,International,Canada,Deplaned,Other,Terminal 1,B,35156,Deplaned,35156,2005,July
4,200507,Air Canada,AC,Air Canada,AC,International,Canada,Enplaned,Other,Terminal 1,B,34090,Enplaned,34090,2005,July


In [ ]:
# Fill null values in 'Operating Airline' with the most frequent value
df['Operating Airline'].fillna(df['Operating Airline'].mode()[0], inplace=True)

# Fill null values in 'GEO Summary' with 'Unknown'
df['GEO Summary'].fillna('Unknown', inplace=True)

# Fill null values in 'GEO Region' with 'Unknown'
df['GEO Region'].fillna('Unknown', inplace=True)

# Verify that there are no more null values
print(df.isnull().sum())


Activity Period                 0
Operating Airline               0
Operating Airline IATA Code    54
Published Airline               0
Published Airline IATA Code    54
GEO Summary                     0
GEO Region                      0
Activity Type Code              0
Price Category Code             0
Terminal                        0
Boarding Area                   0
Passenger Count                 0
Adjusted Activity Type Code     0
Adjusted Passenger Count        0
Year                            0
Month                           0
dtype: int64


In [ ]:
df1 = df.dropna()

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

def aggregate_data(input_0: pd.DataFrame) -> pd.DataFrame:
    grouped_data = input_0.groupby(['Year', 'Month', 'GEO Region'])['Passenger Count'].sum().reset_index()
    grouped_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']
    return grouped_data

def prepare_data(aggregated_data: pd.DataFrame) -> pd.DataFrame:
    label_encoder = LabelEncoder()
    aggregated_data['Year'] = label_encoder.fit_transform(aggregated_data['Year'])
    aggregated_data['Month'] = label_encoder.fit_transform(aggregated_data['Month'])
    aggregated_data['GEO Region'] = label_encoder.fit_transform(aggregated_data['GEO Region'])

    X = aggregated_data.drop(columns=['Total Passenger Count'])
    y = aggregated_data['Total Passenger Count']

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    train_data = pd.concat([X_train, y_train], axis=1)
    test_data = pd.concat([X_test, y_test], axis=1)

    output = pd.concat([train_data, test_data], keys=['train', 'test']).reset_index(level=0).rename(columns={'level_0': 'Set'})

    return output

def train_model(prepared_data: pd.DataFrame) -> RandomForestClassifier:
    feature_cols = ['Year', 'Month', 'GEO Region']
    target_col = 'Total Passenger Count'

    X = prepared_data[feature_cols]
    y = prepared_data[target_col]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    clf = RandomForestClassifier(n_estimators=100, random_state=42)
    clf.fit(X_train, y_train)

    y_pred = clf.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"Model Accuracy: {accuracy}")

    return clf, label_encoder

def predict_availability(clf, label_encoder, year: int, month: int, geo_region: str) -> int:
    year_encoded = label_encoder.transform([year])[0]
    month_encoded = label_encoder.transform([month])[0]
    geo_region_encoded = label_encoder.transform([geo_region])[0]

    input_data = pd.DataFrame([[year_encoded, month_encoded, geo_region_encoded]], columns=['Year', 'Month', 'GEO Region'])

    predicted_count = clf.predict(input_data)[0]

    return predicted_count

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from prophet import Prophet

def aggregate_data(input_0: pd.DataFrame) -> pd.DataFrame:
    input_0['Year'] = input_0['Activity Period'].astype(str).str[:4]
    input_0['Month'] = input_0['Activity Period'].astype(str).str[4:6]
    grouped_data = input_0.groupby(['Year', 'Month', 'GEO Region'])['Passenger Count'].sum().reset_index()
    grouped_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']
    return grouped_data

def create_future_data(aggregated_data: pd.DataFrame) -> pd.DataFrame:
    future_data = []
    geo_regions = aggregated_data['GEO Region'].unique()

    for geo_region in geo_regions:
        df = aggregated_data[aggregated_data['GEO Region'] == geo_region][['Year', 'Month', 'Total Passenger Count']]
        df['ds'] = pd.to_datetime(df['Year'] + df['Month'], format='%Y%m')
        df = df.rename(columns={'Total Passenger Count': 'y'})

        model = Prophet(yearly_seasonality=True)
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        model.fit(df[['ds', 'y']])

        # Calculate the number of months from the last available data point to Dec 2030
        last_date = df['ds'].max()
        future_periods = (2030 - last_date.year) * 12 + (12 - last_date.month)

        future = model.make_future_dataframe(periods=future_periods, freq='M')
        forecast = model.predict(future)

        forecast['Year'] = forecast['ds'].dt.year.astype(str)
        forecast['Month'] = forecast['ds'].dt.month.astype(str).str.zfill(2)
        forecast['GEO Region'] = geo_region
        forecast = forecast[['Year', 'Month', 'GEO Region', 'yhat']]

        future_data.append(forecast)

    future_data = pd.concat(future_data, ignore_index=True)
    future_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']

    return future_data

def prepare_data(aggregated_data: pd.DataFrame) -> (pd.DataFrame, pd.Series, LabelEncoder, LabelEncoder, LabelEncoder):
    year_encoder = LabelEncoder()
    month_encoder = LabelEncoder()
    geo_region_encoder = LabelEncoder()

    aggregated_data['Year'] = year_encoder.fit_transform(aggregated_data['Year'])
    aggregated_data['Month'] = month_encoder.fit_transform(aggregated_data['Month'])
    aggregated_data['GEO Region'] = geo_region_encoder.fit_transform(aggregated_data['GEO Region'])

    X = aggregated_data.drop(columns=['Total Passenger Count'])
    y = aggregated_data['Total Passenger Count']

    return X, y, year_encoder, month_encoder, geo_region_encoder

def train_model(X: pd.DataFrame, y: pd.Series) -> RandomForestRegressor:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)

    print(f"Model Mean Squared Error: {mse}")

    return model

def predict_availability(model, year_encoder, month_encoder, geo_region_encoder, year: int, month: str, geo_region: str) -> (int, bool):
    year_encoded = year_encoder.transform([str(year)])
    month_encoded = month_encoder.transform([month])
    geo_region_encoded = geo_region_encoder.transform([geo_region])

    input_data = pd.DataFrame([[year_encoded[0], month_encoded[0], geo_region_encoded[0]]], columns=['Year', 'Month', 'GEO Region'])

    predicted_count = model.predict(input_data)[0]

    # Define a threshold for flight availability
    threshold = 800  # This value can be adjusted based on your data
    is_available = predicted_count >= threshold

    return predicted_count, is_available

def main():
    # Load your data
    air_traffic_data = pd.read_csv('/kaggle/input/air-traffic-data/air_traffic_data.csv')

    # Aggregate and prepare the data
    aggregated_data = aggregate_data(air_traffic_data)
    future_data = create_future_data(aggregated_data)
    extended_data = pd.concat([aggregated_data, future_data], ignore_index=True)
    X, y, year_encoder, month_encoder, geo_region_encoder = prepare_data(extended_data)

    # Train the model
    model = train_model(X, y)

    # Take user input
    year = int(input(f"Enter the year (between 2005 and 2030): "))
    month = input(f"Enter the month (one of {list(month_encoder.classes_)}): ")
    geo_region = input(f"Enter the geographical region (one of {list(geo_region_encoder.classes_)}): ")

    # Validate user input
    if str(year) not in year_encoder.classes_ or month not in month_encoder.classes_ or geo_region not in geo_region_encoder.classes_:
        print("Invalid input. Please make sure the values are within the specified range.")
        return

    # Predict flight availability
    predicted_count, is_available = predict_availability(model, year_encoder, month_encoder, geo_region_encoder, year, month, geo_region)
    availability_status = "available" if is_available else "not available"
    print(f"Predicted Passenger Count: {predicted_count}")
    print(f"Flight is {availability_status}")

if __name__ == "__main__":
    main()


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/4d4_6y0v.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/fvxd1f3j.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=32722', 'data', 'file=/tmp/tmp8gmzghnx/4d4_6y0v.json', 'init=/tmp/tmp8gmzghnx/fvxd1f3j.json', 'output', 'file=/tmp/tmp8gmzghnx/prophet_model99g2xnv9/prophet_model-20240722173238.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:32:38 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:32:38 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonalit

Model Mean Squared Error: 1371297391.2688375
Enter the year (between 2005 and 2030): 2026
Enter the month (one of ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']): 09
Enter the geographical region (one of ['Asia', 'Australia / Oceania', 'Canada', 'Central America', 'Europe', 'Mexico', 'Middle East', 'South America', 'US']): US
Predicted Passenger Count: 4675435.079568313
Flight is available


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score
from prophet import Prophet

def aggregate_data(input_0: pd.DataFrame) -> pd.DataFrame:
    input_0['Year'] = input_0['Activity Period'].astype(str).str[:4]
    input_0['Month'] = input_0['Activity Period'].astype(str).str[4:6]
    grouped_data = input_0.groupby(['Year', 'Month', 'GEO Region'])['Passenger Count'].sum().reset_index()
    grouped_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']
    return grouped_data

def create_future_data(aggregated_data: pd.DataFrame) -> pd.DataFrame:
    future_data = []
    geo_regions = aggregated_data['GEO Region'].unique()

    for geo_region in geo_regions:
        df = aggregated_data[aggregated_data['GEO Region'] == geo_region][['Year', 'Month', 'Total Passenger Count']]
        df['ds'] = pd.to_datetime(df['Year'] + df['Month'], format='%Y%m')
        df = df.rename(columns={'Total Passenger Count': 'y'})

        model = Prophet(yearly_seasonality=True)
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        model.fit(df[['ds', 'y']])

        # Calculate the number of months from the last available data point to Dec 2030
        last_date = df['ds'].max()
        future_periods = (2030 - last_date.year) * 12 + (12 - last_date.month)

        future = model.make_future_dataframe(periods=future_periods, freq='M')
        forecast = model.predict(future)

        forecast['Year'] = forecast['ds'].dt.year.astype(str)
        forecast['Month'] = forecast['ds'].dt.month.astype(str).str.zfill(2)
        forecast['GEO Region'] = geo_region
        forecast = forecast[['Year', 'Month', 'GEO Region', 'yhat']]

        future_data.append(forecast)

    future_data = pd.concat(future_data, ignore_index=True)
    future_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']

    return future_data

def prepare_data(aggregated_data: pd.DataFrame) -> (pd.DataFrame, pd.Series, LabelEncoder, LabelEncoder, LabelEncoder):
    year_encoder = LabelEncoder()
    month_encoder = LabelEncoder()
    geo_region_encoder = LabelEncoder()

    aggregated_data['Year'] = year_encoder.fit_transform(aggregated_data['Year'])
    aggregated_data['Month'] = month_encoder.fit_transform(aggregated_data['Month'])
    aggregated_data['GEO Region'] = geo_region_encoder.fit_transform(aggregated_data['GEO Region'])

    X = aggregated_data.drop(columns=['Total Passenger Count'])
    y = aggregated_data['Total Passenger Count']

    return X, y, year_encoder, month_encoder, geo_region_encoder

def train_model(X: pd.DataFrame, y: pd.Series) -> RandomForestRegressor:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    model = RandomForestRegressor(n_estimators=100, random_state=42)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)

    print(f"Model Mean Squared Error: {mse}")
    print(f"Model R-squared: {r2}")

    return model, mse, r2

def predict_availability(model, year_encoder, month_encoder, geo_region_encoder, year: int, month: str, geo_region: str) -> (int, bool):
    year_encoded = year_encoder.transform([str(year)])
    month_encoded = month_encoder.transform([month])
    geo_region_encoded = geo_region_encoder.transform([geo_region])

    input_data = pd.DataFrame([[year_encoded[0], month_encoded[0], geo_region_encoded[0]]], columns=['Year', 'Month', 'GEO Region'])

    predicted_count = model.predict(input_data)[0]

    # Define a threshold for flight availability
    threshold = 800  # This value can be adjusted based on your data
    is_available = predicted_count >= threshold

    return predicted_count, is_available

def main():
    # Load your data
    air_traffic_data = pd.read_csv('/kaggle/input/air-traffic-data/air_traffic_data.csv')

    # Aggregate and prepare the data
    aggregated_data = aggregate_data(air_traffic_data)
    future_data = create_future_data(aggregated_data)
    extended_data = pd.concat([aggregated_data, future_data], ignore_index=True)
    X, y, year_encoder, month_encoder, geo_region_encoder = prepare_data(extended_data)

    # Train the model
    model, mse, r2 = train_model(X, y)

    # Print model accuracy metrics
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")

    # Take user input
    year = int(input(f"Enter the year (between 2005 and 2030): "))
    month = input(f"Enter the month (one of {list(month_encoder.classes_)}): ")
    geo_region = input(f"Enter the geographical region (one of {list(geo_region_encoder.classes_)}): ")

    # Validate user input
    if str(year) not in year_encoder.classes_ or month not in month_encoder.classes_ or geo_region not in geo_region_encoder.classes_:
        print("Invalid input. Please make sure the values are within the specified range.")
        return

    # Predict flight availability
    predicted_count, is_available = predict_availability(model, year_encoder, month_encoder, geo_region_encoder, year, month, geo_region)
    availability_status = "available" if is_available else "not available"
    print(f"Predicted Passenger Count: {predicted_count}")
    print(f"Flight is {availability_status}")

if __name__ == "__main__":
    main()


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/tbct9t0p.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/i326w9_g.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=42521', 'data', 'file=/tmp/tmp8gmzghnx/tbct9t0p.json', 'init=/tmp/tmp8gmzghnx/i326w9_g.json', 'output', 'file=/tmp/tmp8gmzghnx/prophet_modelvloq5e2y/prophet_model-20240722173327.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:33:27 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:33:27 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonalit

Model Mean Squared Error: 1371297391.2688375
Model R-squared: 0.9987325330217871
Mean Squared Error: 1371297391.2688375
R-squared: 0.9987325330217871
Enter the year (between 2005 and 2030): 2028
Enter the month (one of ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']): 10
Enter the geographical region (one of ['Asia', 'Australia / Oceania', 'Canada', 'Central America', 'Europe', 'Mexico', 'Middle East', 'South America', 'US']): US
Predicted Passenger Count: 4664996.923334601
Flight is available


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.metrics import mean_squared_error
import numpy as np
from prophet import Prophet

def aggregate_data(input_0: pd.DataFrame) -> pd.DataFrame:
    input_0['Year'] = input_0['Activity Period'].astype(str).str[:4]
    input_0['Month'] = input_0['Activity Period'].astype(str).str[4:6]
    grouped_data = input_0.groupby(['Year', 'Month', 'GEO Region'])['Passenger Count'].sum().reset_index()
    grouped_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']
    return grouped_data

def create_future_data(aggregated_data: pd.DataFrame) -> pd.DataFrame:
    future_data = []
    geo_regions = aggregated_data['GEO Region'].unique()

    for geo_region in geo_regions:
        df = aggregated_data[aggregated_data['GEO Region'] == geo_region][['Year', 'Month', 'Total Passenger Count']]
        df['ds'] = pd.to_datetime(df['Year'] + df['Month'], format='%Y%m')
        df = df.rename(columns={'Total Passenger Count': 'y'})

        model = Prophet(yearly_seasonality=True)
        model.add_seasonality(name='monthly', period=30.5, fourier_order=5)
        model.fit(df[['ds', 'y']])

        # Calculate the number of months from the last available data point to Dec 2030
        last_date = df['ds'].max()
        future_periods = (2030 - last_date.year) * 12 + (12 - last_date.month)

        future = model.make_future_dataframe(periods=future_periods, freq='M')
        forecast = model.predict(future)

        forecast['Year'] = forecast['ds'].dt.year.astype(str)
        forecast['Month'] = forecast['ds'].dt.month.astype(str).str.zfill(2)
        forecast['GEO Region'] = geo_region
        forecast = forecast[['Year', 'Month', 'GEO Region', 'yhat']]

        future_data.append(forecast)

    future_data = pd.concat(future_data, ignore_index=True)
    future_data.columns = ['Year', 'Month', 'GEO Region', 'Total Passenger Count']

    return future_data

def prepare_data(aggregated_data: pd.DataFrame) -> (pd.DataFrame, pd.Series, LabelEncoder, LabelEncoder, LabelEncoder, StandardScaler):
    year_encoder = LabelEncoder()
    month_encoder = LabelEncoder()
    geo_region_encoder = LabelEncoder()
    scaler = StandardScaler()

    aggregated_data['Year'] = year_encoder.fit_transform(aggregated_data['Year'])
    aggregated_data['Month'] = month_encoder.fit_transform(aggregated_data['Month'])
    aggregated_data['GEO Region'] = geo_region_encoder.fit_transform(aggregated_data['GEO Region'])

    X = aggregated_data.drop(columns=['Total Passenger Count'])
    y = aggregated_data['Total Passenger Count']

    X_scaled = scaler.fit_transform(X)

    return X_scaled, y, year_encoder, month_encoder, geo_region_encoder, scaler

def train_model(X: pd.DataFrame, y: pd.Series) -> RandomForestRegressor:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

    param_grid = {
        'n_estimators': [100, 200, 300],
        'max_depth': [None, 10, 20, 30],
        'min_samples_split': [2, 5, 10],
        'min_samples_leaf': [1, 2, 4],
        'bootstrap': [True, False]
    }

    grid_search = GridSearchCV(estimator=RandomForestRegressor(random_state=42), param_grid=param_grid, cv=3, n_jobs=-1, verbose=2)
    grid_search.fit(X_train, y_train)

    best_model = grid_search.best_estimator_

    y_pred = best_model.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = best_model.score(X_test, y_test)

    print(f"Best Model Parameters: {grid_search.best_params_}")
    print(f"Model Mean Squared Error: {mse}")
    print(f"Model R-squared: {r2}")

    return best_model, mse, r2

def predict_availability(model, year_encoder, month_encoder, geo_region_encoder, scaler, year: int, month: str, geo_region: str) -> (int, bool):
    year_encoded = year_encoder.transform([str(year)])
    month_encoded = month_encoder.transform([month])
    geo_region_encoded = geo_region_encoder.transform([geo_region])

    input_data = pd.DataFrame([[year_encoded[0], month_encoded[0], geo_region_encoded[0]]], columns=['Year', 'Month', 'GEO Region'])
    input_data_scaled = scaler.transform(input_data)

    predicted_count = model.predict(input_data_scaled)[0]

    # Define a threshold for flight availability
    threshold = 800  # This value can be adjusted based on your data
    is_available = predicted_count >= threshold

    return predicted_count, is_available

def main():
    # Load your data
    air_traffic_data = pd.read_csv('/kaggle/input/air-traffic-data/air_traffic_data.csv')

    # Aggregate and prepare the data
    aggregated_data = aggregate_data(air_traffic_data)
    future_data = create_future_data(aggregated_data)
    extended_data = pd.concat([aggregated_data, future_data], ignore_index=True)
    X, y, year_encoder, month_encoder, geo_region_encoder, scaler = prepare_data(extended_data)

    # Train the model
    model, mse, r2 = train_model(X, y)

    # Print model accuracy metrics
    print(f"Mean Squared Error: {mse}")
    print(f"R-squared: {r2}")

    # Take user input
    year = int(input(f"Enter the year (between 2005 and 2030): "))
    month = input(f"Enter the month (one of {list(month_encoder.classes_)}): ")
    geo_region = input(f"Enter the geographical region (one of {list(geo_region_encoder.classes_)}): ")

    # Validate user input
    if str(year) not in year_encoder.classes_ or month not in month_encoder.classes_ or geo_region not in geo_region_encoder.classes_:
        print("Invalid input. Please make sure the values are within the specified range.")
        return

    # Predict flight availability
    predicted_count, is_available = predict_availability(model, year_encoder, month_encoder, geo_region_encoder, scaler, year, month, geo_region)
    availability_status = "available" if is_available else "not available"
    print(f"Predicted Passenger Count: {predicted_count}")
    print(f"Flight is {availability_status}")

if __name__ == "__main__":
    main()


INFO:prophet:Disabling weekly seasonality. Run prophet with weekly_seasonality=True to override this.
INFO:prophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/3ue918mg.json
DEBUG:cmdstanpy:input tempfile: /tmp/tmp8gmzghnx/hvcg8t8g.json
DEBUG:cmdstanpy:idx 0
DEBUG:cmdstanpy:running CmdStan, num_threads: None
DEBUG:cmdstanpy:CmdStan args: ['/usr/local/lib/python3.10/dist-packages/prophet/stan_model/prophet_model.bin', 'random', 'seed=32948', 'data', 'file=/tmp/tmp8gmzghnx/3ue918mg.json', 'init=/tmp/tmp8gmzghnx/hvcg8t8g.json', 'output', 'file=/tmp/tmp8gmzghnx/prophet_modelubm5br0a/prophet_model-20240722173447.csv', 'method=optimize', 'algorithm=lbfgs', 'iter=10000']
17:34:47 - cmdstanpy - INFO - Chain [1] start processing
INFO:cmdstanpy:Chain [1] start processing
17:34:47 - cmdstanpy - INFO - Chain [1] done processing
INFO:cmdstanpy:Chain [1] done processing
INFO:prophet:Disabling weekly seasonalit

Fitting 3 folds for each of 216 candidates, totalling 648 fits
Best Model Parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
Model Mean Squared Error: 1373823245.2598524
Model R-squared: 0.9987301984176773
Mean Squared Error: 1373823245.2598524
R-squared: 0.9987301984176773
Enter the year (between 2005 and 2030): 2029
Enter the month (one of ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']): 06
Enter the geographical region (one of ['Asia', 'Australia / Oceania', 'Canada', 'Central America', 'Europe', 'Mexico', 'Middle East', 'South America', 'US']): US
Predicted Passenger Count: 5065453.686835364
Flight is available
